### Importar y cargar datos

In [12]:
import numpy as np
import pandas as pd
import os
# Lista de posibles rutas
addresses = [
    'C:/Users/RONALD Q/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/IEXCITACION.xlsx',
    'C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/IEXCITACION.xlsx',
    'C:/Users/mticllacu/OneDrive - LUZ DEL SUR S.A.A/Archivos de Ronald Quispe Ocaña - ProyectoRyD_V2/Basededatos/IEXCITACION.xlsx'
]

df = None
for path in addresses:
    if os.path.exists(path):   # verifica si existe
        df = pd.read_excel(path,header=1)
        print(f"✅ Archivo cargado desde: {path}")
        break

if df is None:
    raise FileNotFoundError("❌ No se encontró el archivo en ninguna de las rutas especificadas.")
df.head()

✅ Archivo cargado desde: C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/IEXCITACION.xlsx


,Unnamed: 0,SERIE,FECHA DE MUESTRA,R POS1 mA,R POS2 mA,R POS3 mA,R POS4 mA,R POS5 mA,R POS6 mA,R POS7 mA,...,T POS20 mA,T POS21 mA,T POS22 mA,T POS23 mA,T POS24 mA,T POS25 mA,T POS26 mA,T POS27 mA,TempR,HumR
0,NaN,59571,2016-10-23,NaN,68.167,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,59571,2006-01-29,61.932,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,59571,1996-04-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,59572,2016-09-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,59572,2006-01-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Limpieza de datos

In [13]:
# df = df.drop(columns=['Unnamed: 0'])
df = df.rename(columns ={'FECHA DE MUESTRA':'FECHA'})
df = df[['SERIE', 'FECHA'] + [col for col in df.columns if col.endswith("mA")]] # seleccionar columnas relevantes
df.head()

,SERIE,FECHA,R POS1 mA,R POS2 mA,R POS3 mA,R POS4 mA,R POS5 mA,R POS6 mA,R POS7 mA,R POS8 mA,...,T POS18 mA,T POS19 mA,T POS20 mA,T POS21 mA,T POS22 mA,T POS23 mA,T POS24 mA,T POS25 mA,T POS26 mA,T POS27 mA
0,59571,2016-10-23,NaN,68.167,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,59571,2006-01-29,61.932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,59571,1996-04-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,59572,2016-09-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,59572,2006-01-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Lógica

In [3]:
# Calcular columnas delta respecto al valor inicial por SERIE en la fecha mínima
# Detectar columnas numéricas a procesar (excluyendo SERIE y FECHA)
cols_val = [c for c in df.columns if c not in ['SERIE', 'FECHA'] and np.issubdtype(df[c].dtype, np.number)]

for col in cols_val:
    ref = df.groupby('SERIE').apply(lambda g: g.loc[g['FECHA'].idxmin(), col])
    ref_mapped = df['SERIE'].map(ref)
    delta = abs((df[col] - ref_mapped) / ref_mapped) * 100
    delta = delta.replace([np.inf, -np.inf], np.nan)
    mask = df[col].isna() | ref_mapped.isna() | (ref_mapped == 0) | (df[col] == 0)
    delta[mask] = np.nan
    df[f'{col}_Delta'] = delta
df
# Calcular el máximo de las columnas que terminan en 'Delta' para cada fila
cols_delta = [col for col in df.columns if col.endswith('Delta')]
df['Max_Iex'] = df[cols_delta].max(axis=1)

C:\Users\roquispec\AppData\Local\Temp\ipykernel_20824\537266828.py:6: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ref = df.groupby('SERIE').apply(lambda g: g.loc[g['FECHA'].idxmin(), col])
C:\Users\roquispec\AppData\Local\Temp\ipykernel_20824\537266828.py:6: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ref = df.groupby('SERIE').apply(lambda g: g.loc[g['FECHA'].idxmin(), col])
C:\Users\roquispec\AppData\Local\Temp\

### Asignación de Puntaje

In [4]:
# Asignar puntaje IEX según la tabla SD MYERS
# Si Max_Iex > 10% → 5, si Max_Iex ≤ 10% → 1
df['IEX'] = np.where(df['Max_Iex'] > 10, 5, 1)
# Crear tabla final solo con SERIE, FECHA, IEX y las columnas que no terminan en 'Delta' ni son 'Max_Iex'
cols_final = [col for col in df.columns if not col.endswith('Delta') and col != 'Max_Iex']
# Aseguramos el orden: SERIE, FECHA, IEX primero
cols_final = ['SERIE', 'FECHA', 'IEX'] + [col for col in cols_final if col not in ['SERIE', 'FECHA', 'IEX']]
df_final = df[cols_final]

### Extensión de Calendario

In [5]:
# =============================
# TABLAS FINALES Y FUNCIONES
# =============================
# 1. Tabla con fechas originales: SERIE, FECHA, IEX
cols_base = [col for col in df_final.columns if col in ['SERIE', 'FECHA', 'IEX']]
df_IEX = df_final[cols_base].copy()

# 2. Tabla con fechas extendidas: SERIE, FECHA, IEX
inicio = "2015-01-01"
fecha_inicio = pd.Timestamp(inicio)
fecha_fin = pd.Timestamp.today().normalize()
fechas = pd.date_range(fecha_inicio, fecha_fin, freq="D")
todas_series = df['SERIE'].dropna().unique()
calendario = pd.MultiIndex.from_product([todas_series, fechas], names=["SERIE", "FECHA"])
df_calendario = pd.DataFrame(index=calendario).reset_index()

ultimos_2014 = df_IEX[df_IEX['FECHA'] < fecha_inicio].sort_values('FECHA').groupby('SERIE').tail(1)
ultimos_2014['FECHA'] = fecha_inicio
base_ext = pd.concat([df_IEX, ultimos_2014], ignore_index=True)
df_IEX_ext = pd.merge(df_calendario, base_ext, on=["SERIE","FECHA"], how="left")
df_IEX_ext = df_IEX_ext.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)

# 3. Tabla de detalles con fechas originales
cols_detalles = [col for col in df_final.columns]
cols_detalles = ['SERIE', 'FECHA', 'IEX'] + [col for col in cols_detalles if col not in ['SERIE', 'FECHA', 'IEX']]
df_detalles = df_final[cols_detalles].copy()

# 4. Tabla de detalles con fechas extendidas
ultimos_2014_det = df_detalles[df_detalles['FECHA'] < fecha_inicio].sort_values('FECHA').groupby('SERIE').tail(1)
ultimos_2014_det['FECHA'] = fecha_inicio
base_ext_det = pd.concat([df_detalles, ultimos_2014_det], ignore_index=True)
df_detalles_ext = pd.merge(df_calendario, base_ext_det, on=["SERIE","FECHA"], how="left")
df_detalles_ext = df_detalles_ext.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)


C:\Users\roquispec\AppData\Local\Temp\ipykernel_20824\2898523022.py:21: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_IEX_ext = df_IEX_ext.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)
C:\Users\roquispec\AppData\Local\Temp\ipykernel_20824\2898523022.py:21: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_IEX_ext = df_IEX_ext.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)
C:\Users\roquispec\AppData\Local\Temp\ipykernel_20824\2898523022.py:33: 

### Creación de Funciones

In [6]:
# =============================
# FUNCIONES PARA LLAMAR
# =============================
def get_df_IEX():
    return df_IEX

def get_df_extendida_IEX():
    return df_IEX_ext

def get_df_detalles_IEX():
    return df_detalles

def get_df_detalles_ext_IEX():
    return df_detalles_ext

# =============================
# PRINT DE TABLAS
# =============================
print('\n ====== TABLA CON FECHAS ORIGINALES ====== \n')
print(get_df_IEX().head(20), '\n')
print('\n ====== TABLA CON FECHAS EXTENDIDAS ====== \n')
print(get_df_extendida_IEX().tail(), '\n')
print('\n ====== TABLA DE DETALLES CON FECHAS ORIGINALES ====== \n')
print(get_df_detalles_IEX().head(), '\n')
print('\n ====== TABLA DE DETALLES CON FECHAS EXTENDIDAS ====== \n')
print(get_df_detalles_ext_IEX().head(), '\n')


 ====== TABLA CON FECHAS ORIGINALES ====== 

     SERIE      FECHA  IEX
0    59571 2016-10-23    1
1    59571 2006-01-29    1
2    59571 1996-04-16    1
3    59572 2016-09-18    1
4    59572 2006-01-29    1
5    59572 1996-04-22    1
6    59573 2016-09-18    1
7    59573 2006-01-29    1
8   146916 2019-06-30    1
9   146916 2015-01-06    1
10  146916 2014-11-12    1
11  146917 2022-05-12    1
12  146917 2014-12-05    1
13  146918 2019-07-11    5
14  146918 2014-11-25    1
15  185466 2015-11-22    1
16  185466 2014-09-12    1
17  185466 2005-10-21    1
18  185519 2020-06-16    1
19  185519 2016-07-10    1 


 ====== TABLA CON FECHAS EXTENDIDAS ====== 

          SERIE      FECHA  IEX
336598  V101041 2025-10-30  1.0
336599  V101041 2025-10-31  1.0
336600  V101041 2025-11-01  1.0
336601  V101041 2025-11-02  1.0
336602  V101041 2025-11-03  1.0 


 ====== TABLA DE DETALLES CON FECHAS ORIGINALES ====== 

   SERIE      FECHA  IEX  R POS1 mA  R POS2 mA  R POS3 mA  R POS4 mA  \
0  59571 2016-1